In [30]:
from dotenv import load_dotenv, find_dotenv
from pydantic import BaseModel, field_validator, ValidationError
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langgraph.graph import START, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition

In [5]:
find_dotenv()
load_dotenv()

True

In [34]:
# define the Tools
def add(a: int, b: int) -> int:
    """add a and b together

    Args:
        a : first int 
        b : second int

    """
    return a+b


def multiply(a: int, b: int) -> int:
    """multiply a and b

    Args:
        a : first int 
        b : second int


    """
    return a*b


def divide(a: float, b: float) -> float:
    """divide a and b

    Args:
        a : first int 
        b : second int

    """
    return a/b

tools = [add, multiply, divide]
llm = ChatOllama(model="llama3.1:latest")
llm_with_tools = llm.bind_tools(tools)

In [31]:
# Def the nodes
def llm_node(state:MessagesState):
    """defines the assistant Node. returns a Dict with the stae key and the 
    llm invke with the previous state

    Args:
        state: takes in a state as CustomState 
    """
    return {"messages":llm_with_tools.invoke(state["messages"])}

In [32]:
# Def the Graph
builder = StateGraph(MessagesState)
memory = MemorySaver()

builder.add_node("llm_node", llm_node)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "llm_node")
builder.add_conditional_edges("llm_node", tools_condition)
builder.add_edge("tools", "llm_node")

graph = builder.compile(checkpointer=memory)

In [45]:
messages = [HumanMessage(content="Add 3 and 4.")]
config = {"configurable": {"thread_id": "1"}}
graph.invoke({"messages": messages}, config)


TypeError: list indices must be integers or slices, not str

In [36]:
messages = [HumanMessage(content="multiply by 234324")]
config = {"configurable": {"thread_id": "1"}}
graph.invoke({"messages": messages}, config)

{'messages': [HumanMessage(content='Add 3 and 4.', id='3606e010-4d96-4a58-80d4-2fb2a1c334fd'),
  HumanMessage(content='Add 3 and 4.', id='3bb79eff-3eae-4296-8c06-61d9e6acb907'),
  AIMessage(content='', response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-09-27T15:20:21.099234Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'add', 'arguments': {'a': 3, 'b': 4}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 5244020000, 'load_duration': 3100870584, 'prompt_eval_count': 293, 'prompt_eval_duration': 1387798000, 'eval_count': 22, 'eval_duration': 750387000}, id='run-a8b8090f-9775-48ca-b2b1-01054f5e0f8f-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 4}, 'id': 'a6ab0453-eea6-462b-a611-8cb79ddf5f98', 'type': 'tool_call'}], usage_metadata={'input_tokens': 293, 'output_tokens': 22, 'total_tokens': 315}),
  ToolMessage(content='7', name='add', id='1170941f-958f-474e-a515-d7b8da7391e9', tool_call_id='a6ab0453-eea6-462b-a611-8

In [37]:
messages = [HumanMessage(content="divide this by 345")]
config = {"configurable": {"thread_id": "1"}}
graph.invoke({"messages": messages}, config)

{'messages': [HumanMessage(content='Add 3 and 4.', id='3606e010-4d96-4a58-80d4-2fb2a1c334fd'),
  HumanMessage(content='Add 3 and 4.', id='3bb79eff-3eae-4296-8c06-61d9e6acb907'),
  AIMessage(content='', response_metadata={'model': 'llama3.1:latest', 'created_at': '2024-09-27T15:20:21.099234Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'add', 'arguments': {'a': 3, 'b': 4}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 5244020000, 'load_duration': 3100870584, 'prompt_eval_count': 293, 'prompt_eval_duration': 1387798000, 'eval_count': 22, 'eval_duration': 750387000}, id='run-a8b8090f-9775-48ca-b2b1-01054f5e0f8f-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 4}, 'id': 'a6ab0453-eea6-462b-a611-8cb79ddf5f98', 'type': 'tool_call'}], usage_metadata={'input_tokens': 293, 'output_tokens': 22, 'total_tokens': 315}),
  ToolMessage(content='7', name='add', id='1170941f-958f-474e-a515-d7b8da7391e9', tool_call_id='a6ab0453-eea6-462b-a611-8

In [44]:
type(messages)

list

In [49]:
print(messages[0]["HumanMessage"])

TypeError: 'HumanMessage' object is not subscriptable